In [1]:
!pip3 install minio

  Obtaining dependency information for minio from https://files.pythonhosted.org/packages/9a/ae/c20305102e32f078ec5e77a5010378318307a2de213f61be6c5facb56f0f/minio-7.1.16-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00


In [2]:
%%bash
## create dummy files (sleep 1 to generated random string)
echo `date| md5sum` > /tmp/file01 && sleep 1
echo `date| md5sum` > /tmp/file02 && sleep 1
echo `date| md5sum` > /tmp/file03 && sleep 1
echo `date| md5sum` > /tmp/file04 && sleep 1
echo `date| md5sum` > /tmp/file05
cat /tmp/file0*

2c4ed3c806fb15a8156d8e9fc23204df -
f89b7ce5bb4a4e4e47feee0431370e4f -
6488ae1d3a63d8c395742707122e3ba3 -
b388f2274b7296a52f2b956691eec2a6 -
14cdcd085bc3dce79e226c01745fbd83 -


In [17]:
minio_url= "minio-service.kubeflow.svc.cluster.local:9000"
print("minio url:", minio_url)

minio url: minio-service.kubeflow.svc.cluster.local:9000


In [13]:
from minio import Minio
from minio.error import S3Error

BUCKET_NAME="datapipeline-021"

config = {
    "endpoint": minio_url,
    "access_key": "minio",
    "secret_key": "minio123",
    "secure": False,
    }


# Create a client with the MinIO server playground, its access key
# and secret key.
print ("connecting to minio {}".format(minio_url))
minio_client = Minio(**config)

print("try to find bucket {}".format(BUCKET_NAME))
found = minio_client.bucket_exists(BUCKET_NAME)
print("found", found)
if not found:
    minio_client.make_bucket(BUCKET_NAME)
else:
    print("Bucket '{}' already exists".format(BUCKET_NAME))

# # Upload '/home/user/Photos/asiaphotos.zip' as object name
# # 'asiaphotos-2015.zip' to bucket 'asiatrip'.
# client.fput_object(
# "asiatrip", "asiaphotos-2015.zip", "/home/user/Photos/asiaphotos.zip",
# )
# print(
# "'/home/user/Photos/asiaphotos.zip' is successfully uploaded as "
# "object 'asiaphotos-2015.zip' to bucket 'asiatrip'."
# )


connecting to minio minio-service.kubeflow.svc.cluster.local:9000
try to find bucket datapipeline-021
found True
Bucket 'datapipeline-021' already exists


In [14]:
import glob
import os
files=glob.glob("/tmp/file0*")
for filepath in files:
    minio_client.fput_object(BUCKET_NAME, os.path.basename(filepath), filepath)

In [15]:

def download_path(filename):
    return "/tmp/minio_download_{}".format(filename)

for item in minio_client.list_objects(BUCKET_NAME,recursive=False):
    print(item.object_name)
    obj=minio_client.fget_object(BUCKET_NAME,item.object_name,download_path(item.object_name))
    print(obj)


file01
file02
file03
file04
file05


In [16]:
%%bash
for f in `ls /tmp/minio_download*`; do
    echo $f
    cat $f
done


/tmp/minio_download_file01
2c4ed3c806fb15a8156d8e9fc23204df -
/tmp/minio_download_file02
f89b7ce5bb4a4e4e47feee0431370e4f -
/tmp/minio_download_file03
6488ae1d3a63d8c395742707122e3ba3 -
/tmp/minio_download_file04
b388f2274b7296a52f2b956691eec2a6 -
/tmp/minio_download_file05
14cdcd085bc3dce79e226c01745fbd83 -
